In [3]:
from selenium import webdriver
import time, json

COOKIES_FILE = "cookies.json"

driver = webdriver.Chrome()
driver.get("https://vn.trip.com/")  # mở Trip.com trang chủ

print("➡️ Hãy đăng nhập thủ công (Email/Google/Apple).")
print("⏳ Bạn có 60 giây để login...")

time.sleep(60)  # cho bạn login bằng tay

# Sau khi login xong -> lưu cookies ra file
with open(COOKIES_FILE, "w", encoding="utf-8") as f:
    json.dump(driver.get_cookies(), f)

print("✅ Cookies đã được lưu vào", COOKIES_FILE)
driver.quit()


➡️ Hãy đăng nhập thủ công (Email/Google/Apple).
⏳ Bạn có 60 giây để login...
✅ Cookies đã được lưu vào cookies.json


### Main code

In [1]:
# tripcom_crawl_6weeks.py
import os
import time
import json
import pandas as pd
from datetime import datetime, timedelta
from openpyxl import load_workbook

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# -------------------- CONFIG --------------------
COOKIES_FILE = "cookies.json"                    # file cookies bạn đã lưu bằng script login riêng
INPUT_FILE = "Competitor Tracking.xlsx"         # file input, sheet "Hotel Link"
SHEET_NAME = "Hotel Link"
BACKUP_FILE = "tripcom_prices_temp.xlsx"
FINAL_PREFIX = "tripcom_prices_"
TIMEOUT = 20
# ------------------------------------------------

def extract_hyperlinks_from_xlsx(file_path, sheet_name):
    wb = load_workbook(filename=file_path, data_only=True)
    ws = wb[sheet_name]

    urls, hotel_names, room_types = [], [], []
    for row in ws.iter_rows(min_row=2, max_col=2):
        hotel_cell = row[0]
        room_cell = row[1]
        hotel_names.append(hotel_cell.value)
        if hotel_cell.hyperlink:
            urls.append(hotel_cell.hyperlink.target)
        else:
            urls.append(hotel_cell.value if hotel_cell.value else "")
        room_types.append(room_cell.value if room_cell else "")
    df = pd.DataFrame({"Hotel": urls, "Hotel_name": hotel_names, "Room_type": room_types})
    return df

def start_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def load_cookies_into_driver(driver):
    if not os.path.exists(COOKIES_FILE):
        raise FileNotFoundError(f"Cookies file not found: {COOKIES_FILE}")
    with open(COOKIES_FILE, "r", encoding="utf-8") as f:
        cookies = json.load(f)
    driver.get("https://vn.trip.com/")  # load base domain
    time.sleep(1)
    for c in cookies:
        cookie = {}
        for k in ("name", "value", "domain", "path", "secure", "httpOnly", "expiry"):
            if k in c:
                cookie[k] = c[k]
        try:
            driver.add_cookie(cookie)
        except Exception:
            try:
                cookie2 = cookie.copy()
                cookie2.pop("domain", None)
                driver.add_cookie(cookie2)
            except Exception:
                continue
    driver.refresh()
    time.sleep(2)

# ---------------- date selection helpers ----------------
def select_dates_and_search(driver, checkin_date, checkout_date):
    """
    checkin_date, checkout_date dạng 'YYYY-MM-DD'
    """
    wait = WebDriverWait(driver, 15)

    # Click input check-in để mở calendar
    checkin_input = wait.until(EC.element_to_be_clickable((By.ID, "checkInInput")))
    driver.execute_script("arguments[0].click();", checkin_input)
    time.sleep(1)

    # Parse datetime
    checkin_obj = datetime.strptime(checkin_date, "%Y-%m-%d")
    checkout_obj = datetime.strptime(checkout_date, "%Y-%m-%d")

    ensure_date_visible_and_click(driver, checkin_obj)
    time.sleep(0.5)
    ensure_date_visible_and_click(driver, checkout_obj)
    time.sleep(0.5)

    # Click Search
    try:
        search_btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "button.tripui-online-btn-block")
        ))
        driver.execute_script("arguments[0].click();", search_btn)
        time.sleep(1)

        # 🆕 If Trip.com opened a new tab, switch to it
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[-1])
            print("➡️ Switched to new search result tab")

        print(f"✅ Đã chọn {checkin_date} → {checkout_date} và click Search")
    except:
        print("⚠️ Không tìm thấy nút Search.")
    
    time.sleep(2)

    while len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def ensure_date_visible_and_click(driver, target_date):
    """
    Navigate Trip.com calendar until target_date is visible, then click it.
    Works both forward and backward.
    """
    wait = WebDriverWait(driver, 5)
    label = target_date.strftime("%A, %B %d, %Y").replace(" 0", " ")

    for _ in range(12):  # limit attempts
        try:
            elem = driver.find_element(
                By.XPATH, f"//div[@class='tipWrapper' and contains(@aria-label, '{label}')]"
            )
            driver.execute_script("arguments[0].click();", elem)
            print(f"✅ Picked {label}")
            return True
        except:
            # read current month text
            try:
                current_month = driver.find_element(
                    By.CSS_SELECTOR, ".c-calendar-month-title"
                ).text
                current_dt = datetime.strptime(current_month, "%B %Y")
                target_dt = datetime(target_date.year, target_date.month, 1)
            except:
                current_dt = datetime.today()

            target_dt = datetime(target_date.year, target_date.month, 1)

            # navigate calendar
            if target_dt > current_dt:
                next_btn = driver.find_element(By.CSS_SELECTOR, "span.c-calendar-icon-next-mon")
                driver.execute_script("arguments[0].click();", next_btn)
            else:
                prev_btn = driver.find_element(By.CSS_SELECTOR, "span.c-calendar-icon-prev-mon")
                driver.execute_script("arguments[0].click();", prev_btn)

            time.sleep(0.5)

    print(f"❌ Could not find {label} after 12 tries")
    return False

# ---------------- price extraction ----------------
def find_room_block_and_price(driver, room_type):
    blocks = driver.find_elements(By.CSS_SELECTOR, "div.commonRoomCard__BpNjl")
    target_lower = (room_type or "").strip().lower()
    for block in blocks:
        try:
            title = block.find_element(By.CSS_SELECTOR, "span.commonRoomCard-title__iYBn2").text
            if target_lower in title.strip().lower():
                price = extract_price_from_block(block)
                return title, price
        except Exception:
            continue
    return None, "NA"

def extract_price_from_block(block):
    """
    Ưu tiên lấy giá Total (incl. taxes & fees).
    Nếu không có thì fallback về giá hiển thị chính.
    """
    price_xpaths = [
        # Giá Total (incl. taxes & fees)
        ".//div[contains(@class,'saleRoomItemBox-priceBox-priceExplain')]",
        # Giá hiển thị chính
        ".//div[contains(@class,'saleRoomItemBox-priceBox-displayPrice')]//span",
        ".//span[contains(@class,'displayPrice')]",
    ]
    for xp in price_xpaths:
        try:
            el = block.find_element(By.XPATH, xp)
            txt = el.text.strip()
            if txt:
                return txt
        except Exception:
            continue
    return "NA"

# ---------------- main crawl logic ----------------
def main():
    df_urls = extract_hyperlinks_from_xlsx(INPUT_FILE, SHEET_NAME)
    df_urls["Hotel"] = df_urls["Hotel"].apply(lambda x: x.replace("vn.trip.com", "trip.com"))

    prev_data = {}
    if os.path.exists(BACKUP_FILE):
        print(f"📂 Resume from {BACKUP_FILE}")
        df_prev = pd.read_excel(BACKUP_FILE, dtype=str)
        for _, row in df_prev.fillna("NA").iterrows():
            key = (row["Hotel"], row["Room"])
            prev_data[key] = {f"Price W{i}": row.get(f"Price W{i}", "NA") for i in range(1, 6+1)}

    driver = start_driver()
    try:
        load_cookies_into_driver(driver)
    except FileNotFoundError as e:
        driver.quit()
        raise e

    base_checkin = datetime.today() + timedelta(days=1)
    base_checkout = base_checkin + timedelta(days=1)
    week_offsets = [0, 7, 14, 21, 28, 35]
    all_week_prices = {}

    for idx, row in df_urls.iterrows():
        hotel_url = row["Hotel"]
        hotel_name = row["Hotel_name"] or hotel_url
        room_type = row["Room_type"] or ""
        key = (hotel_name, room_type)
        print(f"\n🏨 Crawling ({idx+1}/{len(df_urls)}): {hotel_name} | Room: {room_type}")

        if not str(hotel_url).startswith("http"):
            continue
        all_week_prices.setdefault(key, {})
        driver.execute_script("window.open(arguments[0], '_blank');", hotel_url)
        
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        for week_num, offset in enumerate(week_offsets, start=1):
            key_prefix = f"Price W{week_num}"
            if key in prev_data and prev_data[key].get(key_prefix) and prev_data[key][key_prefix] != "NA":
                all_week_prices[key][key_prefix] = prev_data[key][key_prefix]
                continue

            found = False
            
            for retry_offset in range(0, 7):
                checkin = (base_checkin + timedelta(days=offset + retry_offset)).strftime("%Y-%m-%d")
                checkout = (base_checkout + timedelta(days=offset + retry_offset)).strftime("%Y-%m-%d")
                
                try:
                    time.sleep(4)
                    select_dates_and_search(driver, checkin, checkout)
                except Exception as e:
                    print(f"Error select dates {checkin} -> {checkout}: {e}")
                
                try:

                    title, price = find_room_block_and_price(driver, room_type)
                    if price != "NA":
                        all_week_prices[key][key_prefix] = price
                        print(f"✅ {key_prefix} | {checkin} → {price}")
                        found = True
                        break
                except Exception as e:
                    print(f"⚠️ Error {title}: {e}")

            if not found:
                all_week_prices[key][key_prefix] = "NA"

            # save temp
            temp_result = {"Hotel": [], "Room": []}
            for i in range(1, 7):
                temp_result[f"Price W{i}"] = []
            for (h, r), prices in all_week_prices.items():
                temp_result["Hotel"].append(h)
                temp_result["Room"].append(r)
                for i in range(1, 7):
                    temp_result[f"Price W{i}"].append(prices.get(f"Price W{i}", "NA"))
            pd.DataFrame(temp_result).to_excel(BACKUP_FILE, index=False)

    # Export final
    result_final = {"Hotel": [], "Room": []}
    for i in range(1, 7):
        result_final[f"Price W{i}"] = []
    for (h, r), prices in all_week_prices.items():
        result_final["Hotel"].append(h)
        result_final["Room"].append(r)
        for i in range(1, 7):
            result_final[f"Price W{i}"].append(prices.get(f"Price W{i}", "NA"))
    final_filename = f"{FINAL_PREFIX}{datetime.today().strftime('%Y%m%d')}.xlsx"
    pd.DataFrame(result_final).to_excel(final_filename, index=False)
    print(f"\n✅ Done. Saved: {final_filename}")
    driver.quit()

if __name__ == "__main__":
    main()



🏨 Crawling (1/6): Halais Hotel | Room: Superior Room
✅ Picked Thursday, October 9, 2025
✅ Picked Friday, October 10, 2025
✅ Đã chọn 2025-10-09 → 2025-10-10 và click Search
✅ Picked Friday, October 10, 2025
✅ Picked Saturday, October 11, 2025
✅ Đã chọn 2025-10-10 → 2025-10-11 và click Search
✅ Price W1 | 2025-10-10 → Total (incl. taxes & fees): VND 952,639
✅ Picked Thursday, October 16, 2025
✅ Picked Friday, October 17, 2025
➡️ Switched to new search result tab
✅ Đã chọn 2025-10-16 → 2025-10-17 và click Search
✅ Picked Friday, October 17, 2025
✅ Picked Saturday, October 18, 2025
✅ Đã chọn 2025-10-17 → 2025-10-18 và click Search
✅ Price W2 | 2025-10-17 → Total (incl. taxes & fees): VND 1,002,778
✅ Picked Thursday, October 23, 2025
✅ Picked Friday, October 24, 2025
✅ Đã chọn 2025-10-23 → 2025-10-24 và click Search
✅ Price W3 | 2025-10-23 → Total (incl. taxes & fees): VND 1,002,778
✅ Picked Thursday, October 30, 2025
✅ Picked Friday, October 31, 2025
✅ Đã chọn 2025-10-30 → 2025-10-31 và c

In [31]:
df_urls = extract_hyperlinks_from_xlsx(INPUT_FILE, SHEET_NAME)
df_urls = df_urls.head(2)

In [35]:
print(df_urls.loc[0, "Hotel"])

https://www.trip.com/hotels/detail/?cityId=286&hotelId=25731039&checkIn=2024-07-17&checkOut=2024-07-18&adult=2&children=0&subStamp=672&crn=1&ages=&travelpurpose=0&curr=VND&link=title&hoteluniquekey=H4sIAAAAAAAAAONaxMTFJMEkxMTBKDWdiePl61f3WC1-CzoygABHtoPn4a8aMf2r8hwCeArBYgdCHRoZlxTYcl1_7OowibGWEyR44pihoyCIEWKh56jkxLF26TYjAUWJt1smT2VUYNQEa2TwcjCEMSw2izrCzPCEMYJYOO43cUexcExcz6kkr8AC0dbgCNUGZFg84wQqWrydPYrBiYXjRK_7DMb7-xfxrGBk3Mgoss79YZWIrcMORqYTjFO36y1genX0As8uJrC6Q0Bq4xy2U0wMl5gYbjExPALym3YJvWJi-MQEtu8XRF0TM0MXM8MkZrDqWcxgRYuYGaR4jcwt01JMElMTTdOMkxWENF40XNjCZmTNAfSdlmB8iGeYn2Fiem6BSaaFqVmmsQHjJEamiIhTjFKG5gZmZuaGBhYWhpZmepaJFqkhuflRacUhjlbMUoxuHoxBbI5Oxm4mplFaXMxhfi6CkBD6YC8F4inCeFognmFzbP2NzxuN7JNYU_N0IyIyXvAXMHYxcggwejBGMFYwvmIEqfoBDhIAahnkA9cBAAA&subChannel=&masterhotelid_tracelogid=279fd4aea5f3c&NewTaxDescForAmountshowtype0=T&detailFilters=17%7C1~17~1*80%7C0%7C1~80~0&hotelType=normal&barcurr=VND&locale=en-XX
